In [120]:
# use this as as base for CNN model
# https://www.kaggle.com/sunnyguha/basic-cnn-starter

In [2]:
# import images - 1000 to begin with and on a lazy basis
#   need 1000 images that correspond to 1000 y_train
#   images as np.array(1000,size,size,3) and np.array(1000,1)
#   user Keras flow from directory

# CNN Model
#   Layers
#   Compile
#     During compile stage, set the parameter that you want to minimize
#     Weighted column


# View results

# Check
#   In the case of false positives for a class, is it more likely that the result is of a different class or that it is negative in all classes


In [2]:
!python --version

Python 2.7.13


In [6]:
import sys
sys.version

'3.5.3 (default, Sep 27 2018, 17:25:39) \n[GCC 6.3.0 20170516]'

In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals #to check for gpu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Images
import pydicom
import cv2

# Batching for DataGenerator
from tensorflow.python.keras.utils.data_utils import Sequence

# Keras and Tensorflow
import tensorflow-gpu as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


print("Keras: ",keras.__version__)
print("Tensorflow: ",tf.__version__)
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

SyntaxError: invalid syntax (<ipython-input-11-a041d8f072f9>, line 15)

In [4]:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16940113509886823645
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 179859568340977389
physical_device_desc: "device: XLA_CPU device"
]


In [6]:
tf.test.is_gpu_available()

False

In [3]:
import tensorflow as tf

c = []
for i, d in enumerate(['/gpu:0', '/gpu:1', '/gpu:2']):
    with tf.device(d):
        a = tf.get_variable(f"a_{i}", [2, 3], initializer=tf.random_uniform_initializer(-1, 1))
        b = tf.get_variable(f"b_{i}", [3, 2], initializer=tf.random_uniform_initializer(-1, 1))
        c.append(tf.matmul(a, b))

with tf.device('/cpu:0'):
    sum = tf.add_n(c)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(sum))

SyntaxError: invalid syntax (<ipython-input-3-a7323e8bc0e7>, line 6)

In [175]:
# The issue has to do with the parameter I load into DataGenerator

# from os import listdir
# from os.path import isfile, join
# import os
# from sklearn.utils import shuffle


# import tensorflow as tf
# import keras

# from tensorflow.python.ops import array_ops
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.utils import to_categorical
# from keras.callbacks import ModelCheckpoint
# from tqdm import tqdm

# import gc

In [176]:
# Set global variables
# un-comment as they are used
batch_size=100
# validation_ratio=0.1
# sample_size=2000
# epochs=3
# img_size=512
data_path = "../data/"   # used when calling DataGenerator
random_gen = np.random.RandomState(seed=12345)  # seed is not working

# dataset size to use during model building. Remove these later.
train_size = 10000
valid_size = 2000

In [177]:
train_raw = pd.read_csv(data_path + 'stage_1_train.csv')
train_raw['filename'] = train_raw['ID'].apply(lambda x: "ID_" + x.split('_')[1] + ".dcm")
train_raw['type'] = train_raw['ID'].apply(lambda x: x.split('_')[2])

train_raw.reset_index(drop=True).head()

,ID,Label,filename,type
0,ID_63eb1e259_epidural,0,ID_63eb1e259.dcm,epidural
1,ID_63eb1e259_intraparenchymal,0,ID_63eb1e259.dcm,intraparenchymal
2,ID_63eb1e259_intraventricular,0,ID_63eb1e259.dcm,intraventricular
3,ID_63eb1e259_subarachnoid,0,ID_63eb1e259.dcm,subarachnoid
4,ID_63eb1e259_subdural,0,ID_63eb1e259.dcm,subdural


In [178]:
train_pivot = train_raw[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()

train_pivot.head()

type,filename,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0.dcm,0,0,0,0,0,0
1,ID_00005679d.dcm,0,0,0,0,0,0
2,ID_00008ce3c.dcm,0,0,0,0,0,0
3,ID_0000950d7.dcm,0,0,0,0,0,0
4,ID_0000aee4b.dcm,0,0,0,0,0,0


In [179]:
# train_pivot.sum()
# extract y_train (class vector)
x_y_train.iloc[:,1:].sum()



any                 97103
epidural             2761
intraparenchymal    32564
intraventricular    23766
subarachnoid        32122
subdural            42496
dtype: int64

In [180]:
# create a new df for y_train to get out of pivot table format
train = pd.DataFrame(
    train_pivot,columns=[
        'filename',
        'any',
        'epidural',
        'intraparenchymal',
        'intraventricular',
        'subarachnoid',
        'subdural'
    ])

# # extract y_train (class vector)
# y_train = x_y_train.iloc[:,1:].copy()

# # extract x_train filenames
# x_train_id = train.filename

Some functions to extract the correct window-image. These have been adapted from https://www.kaggle.com/allunia/rsna-ih-detection-eda and references therin

In [181]:
def window_image(img, window_center,window_width, intercept, slope, rescale=True):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    if rescale:
        # Extra rescaling to 0-1, not in the original notebook
        img = (img - img_min) / (img_max - img_min)
    
    return img
    
def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

DataGenerator class based on code written by https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.  The DataGenerator class inherits from the Keras utils.Sequence class and is used to read and serve the train data in batches.

In [182]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs_labels, data_path = '', batch_size=100, dim=(512,512)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs_labels
        self.data_path = data_path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs['filename'][k] for k in indexes]
        list_label_temp=[[int(self.list_IDs['any'][i]),int(self.list_IDs['epidural'][i]),int(self.list_IDs['intraparenchymal'][i]),int(self.list_IDs['intraventricular'][i]),int(self.list_IDs['subarachnoid'][i]),int(self.list_IDs['subdural'][i])] for i in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp,list_label_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp,list_label_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            ds=pydicom.dcmread(self.data_path+'stage_1_train_images/' +list_IDs_temp[i] )
            temp=ds.pixel_array
            window_center , window_width, intercept, slope = get_windowing(ds)
            img = window_image(temp, 50, 100, intercept, slope)
            resized = cv2.resize(img, (200, 200))
            X.append(resized)       
        X=np.array(X).reshape(-1,200,200,1)
        y_train=np.asarray(list_label_temp) 
        return X,y_train

In [183]:
# replicating a version that works
# can remove later

from sklearn.utils import shuffle
train = shuffle(train_pivot)
train_sample=train
train_sample2=train_sample.reset_index(drop=True)
train_sample2.head()

type,filename,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_35bc62b1e.dcm,0,0,0,0,0,0
1,ID_a3c9515cc.dcm,0,0,0,0,0,0
2,ID_1f7bd56ae.dcm,0,0,0,0,0,0
3,ID_0ad2f7be5.dcm,0,0,0,0,0,0
4,ID_da790a955.dcm,0,0,0,0,0,0


In [184]:
valid=train_sample2[100000:120000]
valid=valid.reset_index(drop=True)
valid.head()

type,filename,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_8b10efc1e.dcm,0,0,0,0,0,0
1,ID_f6e2d3233.dcm,0,0,0,0,0,0
2,ID_897e94a32.dcm,0,0,0,0,0,0
3,ID_66eda233e.dcm,0,0,0,0,0,0
4,ID_67c75ae26.dcm,0,0,0,0,0,0


In [189]:
traingen=DataGenerator(train_sample2[0:100],data_path = data_path)#increase back to 100000
validgen=DataGenerator(valid,data_path = data_path)

In [190]:

# Disabled for the moment - later re-enable
# train_actual = train[0:train_size]
# train_valid = train[train_size:(train_size+valid_size)]

# traingen=DataGenerator(train_actual,data_path = data_path, batch_size=batch_size) #increase back to 100000
# validgen=DataGenerator(train_valid,data_path = data_path, batch_size=batch_size)

For an unbalanced data-set it is good to define a custom loss function like the focal loss. We will borrowed the code for this from https://www.kaggle.com/xhlulu/rsna-intracranial-simple-densenet-in-keras

In [191]:
def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

Now that we have both Input=X , Category=y_trest we can train a basic barebones model.
Here we have a model which is like ==   CNN -> CNN->  Flatten --> Dense ---> Dense(output)

In [192]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(200, 200,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5,5)))

model.add(Flatten())  

# model.add(Dense(100))
# model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.compile(loss=focal_loss,
#                optimizer='adam',
#                metrics=['accuracy'])

# model.fit(X,y_train,batch_size=32,epochs=6,validation_split=0.5)
history=model.fit_generator(
    generator=traingen,
    validation_data=validgen,
    use_multiprocessing=True,
    workers=1,
    epochs=2)

Epoch 1/2
1/1 [==============================] - 220s 220s/step - loss: 0.5895 - accuracy: 0.4400 - val_loss: 0.5915 - val_accuracy: 0.0982
Epoch 2/2
1/1 [==============================] - 113s 113s/step - loss: 0.5324 - accuracy: 0.1100 - val_loss: 0.5832 - val_accuracy: 0.0468


In [ ]:
# What's this error?
# things not yet imported, below

from os import listdir
from os.path import isfile, join
import os
from sklearn.utils import shuffle


import tensorflow as tf
import keras

from tensorflow.python.ops import array_ops
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

import gc


In [ ]:
# plot accuracy

plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# plot loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()